In [ ]:
import zipfile
import os

from google.colab import drive
drive.mount('/content/drive')

zip_path = "/content/drive/My Drive/yakaboo-book-reviews-dataset.zip"
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Архів успішно розпаковано в:", extract_path)

In [ ]:
import zipfile

zip_files = ["reviews.jsonlines.zip", "books.jsonlines.zip"]

for zip_file in zip_files:
    zip_path = os.path.join(extract_path, zip_file) 
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)  
    print(f"{zip_file} розпаковано")

print("Файли після розпакування:", os.listdir(extract_path))

In [ ]:
import pandas as pd
import os

reviews_path = os.path.join(extract_path, "reviews.jsonlines")

In [ ]:
df_reviews = pd.read_json(reviews_path, lines=True)

print(df_reviews.head()) 

In [ ]:
def get_sentiment(rating):
    if rating >= 4:
        return 0   # "POSITIVE"
    else:
        return 1  # "NEGATIVE"

In [ ]:
df_reviews['sentiment'] = df_reviews['rating'].apply(get_sentiment)

print(df_reviews[['review', 'sentiment']])

In [ ]:
sentiment_counts = df_reviews["sentiment"].value_counts()
print(sentiment_counts)

num_positive = (df_reviews["sentiment"] == 0).sum()
num_negative = (df_reviews["sentiment"] == 1).sum()

print(f"Позитивних: {num_positive}")
print(f"Негативних: {num_negative}")


sentiment
0    66200
1     3056
Name: count, dtype: int64
Позитивних: 66200
Негативних: 3056


In [ ]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

df_positive = df_reviews[df_reviews["sentiment"] == 0]
df_negative = df_reviews[df_reviews["sentiment"] == 1]

df_positive_balanced = resample(df_positive,
                                replace=False,
                                n_samples=len(df_negative),
                                random_state=42)

df_balanced = pd.concat([df_positive_balanced, df_negative])

train_reviews, test_reviews, train_sentiment, test_sentiment = train_test_split(
    df_balanced["review"], df_balanced["sentiment"], test_size=0.2, random_state=42
)

print(f"Train size: {len(train_reviews)}, Test size: {len(test_reviews)}")
print(df_balanced["sentiment"].value_counts())

Train size: 4889, Test size: 1223
sentiment
0    3056
1    3056
Name: count, dtype: int64


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Підготовка даних
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,  
            max_length=self.max_len, 
            padding='max_length',  
            truncation=True,  
            return_tensors='pt', 
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


max_len = 50
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_texts = train_reviews.reset_index(drop=True)
val_texts = test_reviews.reset_index(drop=True)
train_labels = train_sentiment.reset_index(drop=True)
val_labels = test_sentiment.reset_index(drop=True)

train_dataset = SentimentDataset(train_texts, train_labels, tokenizer, max_len)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer, max_len)

train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=256)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

optimizer = AdamW(model.parameters(), lr=2e-5)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [ ]:
def train_epoch(model, data_loader, optimizer, device):
    model = model.train()
    losses = []
    correct_predictions = 0

    for batch in tqdm(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        losses.append(loss.item())
        preds = torch.argmax(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)

        loss.backward()
        optimizer.step()

    return correct_predictions.double() / len(data_loader.dataset), sum(losses) / len(data_loader)

def eval_epoch(model, data_loader, device):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for batch in tqdm(data_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            losses.append(loss.item())
            preds = torch.argmax(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    return correct_predictions.double() / len(data_loader.dataset), sum(losses) / len(data_loader)

In [ ]:
for epoch in range(40):
    print(f"Epoch {epoch + 1}")

    train_accuracy, train_loss = train_epoch(model, train_dataloader, optimizer, device)
    print(f"Train loss {train_loss} accuracy {train_accuracy}")

    val_accuracy, val_loss = eval_epoch(model, val_dataloader, device)
    print(f"Validation loss {val_loss} accuracy {val_accuracy}")

Epoch 1


100%|██████████| 20/20 [01:13<00:00,  3.68s/it]


Train loss 0.7002316117286682 accuracy 0.4913070157496421


100%|██████████| 5/5 [00:12<00:00,  2.60s/it]


Validation loss 0.6905019760131836 accuracy 0.5331152902698283
Epoch 2


...

Validation loss 1.8515033006668091 accuracy 0.6197874080130826
Epoch 40


100%|██████████| 20/20 [01:17<00:00,  3.88s/it]


Train loss 0.0468123019207269 accuracy 0.9801595418285949


100%|██████████| 5/5 [00:13<00:00,  2.67s/it]

Validation loss 1.8995713472366333 accuracy 0.6238757154538022


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model_save_path = '/content/drive/MyDrive/bert_model_v2/bert_model.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

tokenizer_save_path = '/content/drive/MyDrive/bert_model_v2/bert_tokenizer'
tokenizer.save_pretrained(tokenizer_save_path)
print(f"Tokenizer saved to {tokenizer_save_path}")